In [1]:
#Import Libriaries
import pandas as pd
import numpy as np
import re
import json

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [ ]:
import paho.mqtt.client as mqtt

df = pd.DataFrame()
print_outcomes = pd.DataFrame(columns=["Predicted Value", "Actual Value"])
message_counter = 0

def on_connect(client, userData, flags, rc):
    print("Connected with the result code: " + str(rc))
    #We subscribe to this data.
    client.subscribe("diabetes/records")
    

def on_message(client, userData, msg):
    global df, print_outcomes, message_counter
    #Decode the incoming message
    data = json.loads(msg.payload.decode())
    message_counter+=1

    #Convert from disctionary
    new_data = pd.DataFrame([data])

    #Append to the existing Data frame
    df = pd.concat([df, new_data], ignore_index=True)
    
    if(len(df)>5):
        outcomes = apply_classification(df.iloc[-5:-1], df.iloc[-1])

        if(len(print_outcomes) == 0):
            print_outcomes = outcomes.copy()
        else:
            print_outcomes = pd.concat([outcomes, print_outcomes],ignore_index=True)

        if(message_counter>=10):
            client.disconnect()
            client.loop_stop()
            correct_predictions = (print_outcomes["Predicted Value"] == print_outcomes["Actual Value"]).sum()
            
            print(print_outcomes)
            print(f"Correct Predictions:{correct_predictions}/{len(print_outcomes)}")
        
   
def apply_classification(data, last_record):
    #Initialize the model
    knn_model = KNeighborsClassifier(n_neighbors=3)

    #Split the data
    X_train = data[["Glucose", "BloodPressure", "Insulin"]]
    y_train = data["Outcome"]
    
    z = last_record[["Glucose", "BloodPressure", "Insulin"]].to_frame().T
    z_actual=last_record["Outcome"]

    #Train the model
    knn_model.fit(X_train, y_train)

    #Make predictions
    y_pred = knn_model.predict(z)[0]

    #Print the predicted value against the the actual value
    outcomes = pd.DataFrame({
        "Predicted Value":[y_pred],
        "Actual Value":[z_actual]
    })

    return outcomes
    
mqttc = mqtt.Client()
mqttc.on_connect = on_connect
mqttc.on_message = on_message
# Now, we connect to the data broker.
mqttc.connect("mqtt.eclipseprojects.io", 1883, 60)

# As a simple example, we just keep the data listening/receiving on and on...
mqttc.loop_forever()

Connected with the result code0
   Predicted Value  Actual Value
0              0.0           0.0
1              1.0           0.0
2              0.0           1.0
3              0.0           0.0
4              0.0           1.0
Correct Predictions:2/5


7